In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
sample_idx = 0
for filename in glob.glob('/home/garner1/pipelines/WSI-analysis/SG/pipeline/data/tcga.detection/*.gz'):
    try:
        if sample_idx == 0:
            df = pd.read_csv(filename,sep='\t')
            features = df.columns[np.r_[0, 7:18]]
            df_pool = df[features].describe().reset_index()
            df_pool['Image'] = [df[df.columns[0]].iloc[0]]*df_pool.shape[0]
        else:
            df = pd.read_csv(filename,sep='\t')
            df_summary = df[features].describe().reset_index()
            df_summary['Image'] = [df[df.columns[0]].iloc[0]]*df_summary.shape[0]
            df_pool = pd.concat([df_pool, df_summary], ignore_index=True)
        print(sample_idx,df_pool.shape)
        sample_idx += 1
    except:
        pass

In [ ]:
import seaborn as sns
sns.set(style="ticks")
df_pool = pd.read_pickle('../data/pooled_measurements_min-25-50-75-max.pkl')
data = df_pool[df_pool['index'].isin(['min','25%','50%','75%','max'])]

In [ ]:
import matplotlib.pyplot as plt
sns.set(font_scale=2)
for feature_id in range(1,12):
    g = sns.catplot(x="Image", y=data.columns[feature_id], data=data, 
                    kind='box',
                    whis=0,
                    height=6, # make the plot 5 units high
                    aspect=6)
    
    ymin = data[data['index']=='25%'][data.columns[feature_id]].min()
    ymin -= ymin/20 
    ymax = data[data['index']=='75%'][data.columns[feature_id]].max()
    ymax += ymax/20 
    
    g.set(ylim=(ymin, ymax),xticks=[])
    plt.savefig(data.columns[feature_id].replace(" ", "").replace(":", ".")+'.png')

In [ ]:
import matplotlib.pyplot as plt
sns.set(font_scale=1)
for feature_id in range(1,12):
    hist = data.hist(column=data.columns[feature_id], by='index', figsize=(10,10),bins=10)
    plt.suptitle(data.columns[feature_id])
    plt.savefig(data.columns[feature_id].replace(" ", "").replace(":", ".")+'.histograms.png')